##Training Model

In [0]:
import os
import random
import glob
import numpy as np
import pandas as pd
import torch
import csv
import torch.nn as nn
import torchvision.models as models
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from torchsummary import summary
import face_recognition

Base_dir = 'data/'

class Resnet18(nn.Module):
    def __init__(self):
        super(Resnet18, self).__init__()
        self.resnet = nn.Sequential(*list(models.resnet18(pretrained=True).children())[:-1])
        self.fc = nn.Linear(512,7)
    def forward(self, x):
        x = self.resnet(x)
        x = x.view(-1, 1*1*512)
        x = self.fc(x)
        
        return x

class hw3_Testdataset(Dataset):
    
    def __init__(self, data, transform):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = Image.open(self.data[idx]).convert('RGB')
        img = self.transform(img)
        return img

def getPredict(test_path, model_path):

  transform = transforms.Compose([transforms.ToTensor()])

  test_img = sorted(glob.glob(os.path.join(test_path, '*.jpg')))
  test_dataset = hw3_Testdataset(test_img,transform)
  test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

  model = Resnet18()
  model.load_state_dict(torch.load(model_path))
  device = torch.device('cuda')
  model = model.to(device)
  model.eval()
  
  prediction = []
  with torch.no_grad():
    for img in test_loader:
      img = img.to(device)
      out = model(img)
      _, pred_label = torch.max(out, 1)
      for i in range(pred_label.size(0)):
        prediction.append(pred_label[i].item())

  df = pd.DataFrame({'id': np.arange(0,len(prediction)), 'label': prediction})
  df.to_csv(Base_dir+'prediction_2.csv',index=False)
  
  return prediction

def getLandmark(test_path):
  
  lands = []
  test_img = sorted(glob.glob(os.path.join(test_path, '*.jpg')))

  for i, name in enumerate(test_img):

  # Load the jpg file into a numpy array
    image = face_recognition.load_image_file(name)

    # Find all facial features in all the faces in the image
    face_landmarks_list = face_recognition.face_landmarks(image)

    #Create a white image to show landmark only
    landmark_img = Image.new('RGB', (48,48), (255,255,255,0))
    pure_d = ImageDraw.Draw(landmark_img)

    for face_landmarks in face_landmarks_list:

        for facial_feature in face_landmarks.keys():
            pure_d.line(face_landmarks[facial_feature], width=1, fill='black')

    landmark_img.save(Base_dir+'Test_landmark/landmark_{}.jpg'.format(i))
    print('{} landmark file number saved'.format(i))
    
    land = np.zeros(1)
    for val in face_landmarks.values():
      nVal = np.array(val).flatten()
      land = np.append(land, nVal)
    land = np.delete(land,0)
    lands.append(land)

  lands = np.array(lands)
  np.save(Base_dir+'Test_landmarks.npy', lands)
  print('Test landmarks shape: ',lands.shape)
  
  return lands

def mixLandmark(emo, ori):
  
  response = np.genfromtxt(Base_dir+'prediction_2.csv', delimiter=',', dtype=int, skip_header=1)[:,1]
  test_land = np.load(Base_dir+'Test_landmarks.npy')
  basic_land = np.load(Base_dir+'remix_landmark/remix_landmark.npy')
  print('basic_land shape: ' , basic_land.shape)
  
  emotion_land = np.zeros((1,144))
  for res in response:
    emotion_land = np.concatenate((emotion_land, basic_land[res].reshape((1,144))))
  emotion_land = np.delete(emotion_land, 0, 0)
  print('emotion landmarks shape: ', emotion_land.shape) # suppose to be (21,144)

  # get the mean landmark
  print('test landmarks shape: ', test_land.shape)
  mix_land = np.around(emotion_land*emo + test_land*ori)
  mix_land = np.array(mix_land, dtype=int)

  # Output the Image
  lines = [17, 5, 5, 4, 5, 6, 6, 12, 12]
  ret = []
  for idx in range(21):

    landmark_img = Image.new('RGB', (48,48), (255,255,255,0))
    pure_d = ImageDraw.Draw(landmark_img)


    base = 0
    for dotnum in lines:
      mark = []
      for k in range(dotnum):
        ptr = base + k*2
        point = (mix_land[idx][ptr], mix_land[idx][ptr+1])
        mark.append(point)
      pure_d.line(mark, width=1, fill='black')
      base += dotnum*2

    #landmark_img.save(Base_dir+'emo_remix/result_{}.jpg'.format(idx))

    #im1 = Image.open(Base_dir+'Testdata/{}.jpg'.format(str(idx).zfill(2))).convert('RGB')
    #im2 = landmark_img

    #combo = Image.fromarray(np.hstack((np.array(im1),np.array(im2))))

    #combo.show()
    #combo.save(Base_dir+'result_landmark/result_landmark_{}.jpg'.format(idx))

    ret.append(landmark_img)
  
  return ret


#prediction = getPredict(Base_dir+'Testdata', Base_dir+'Imgemo_model.pth')
#test_landmarks = getLandmark(Base_dir+'Testdata')
mixLandmark(0.8,0.2)

basic_land shape:  (7, 144)
emotion landmarks shape:  (21, 144)
test landmarks shape:  (21, 144)


[<PIL.Image.Image image mode=RGB size=48x48 at 0x7FB6B1BD8EB8>,
 <PIL.Image.Image image mode=RGB size=48x48 at 0x7FB6B1BD8C50>,
 <PIL.Image.Image image mode=RGB size=48x48 at 0x7FB6B1BD8DA0>,
 <PIL.Image.Image image mode=RGB size=48x48 at 0x7FB6B1BD8C88>,
 <PIL.Image.Image image mode=RGB size=48x48 at 0x7FB6B1BD8E10>,
 <PIL.Image.Image image mode=RGB size=48x48 at 0x7FB6B1BD8CC0>,
 <PIL.Image.Image image mode=RGB size=48x48 at 0x7FB6B1BD8B00>,
 <PIL.Image.Image image mode=RGB size=48x48 at 0x7FB6B1BD8A58>,
 <PIL.Image.Image image mode=RGB size=48x48 at 0x7FB6B1BD89E8>,
 <PIL.Image.Image image mode=RGB size=48x48 at 0x7FB6B1BD88D0>,
 <PIL.Image.Image image mode=RGB size=48x48 at 0x7FB6B1BD8BE0>,
 <PIL.Image.Image image mode=RGB size=48x48 at 0x7FB6B1BD8EF0>,
 <PIL.Image.Image image mode=RGB size=48x48 at 0x7FB6B1BD8F28>,
 <PIL.Image.Image image mode=RGB size=48x48 at 0x7FB6B1BD8F98>,
 <PIL.Image.Image image mode=RGB size=48x48 at 0x7FB6B1BDE048>,
 <PIL.Image.Image image mode=RGB size=48

In [0]:
ret1 = mixLandmark(0.6, 0.4)
ret2 = mixLandmark(0.8, 0.2)

for i in range(21):
    im1 = Image.open(Base_dir+'Testdata/{}.jpg'.format(str(i).zfill(2))).convert('RGB')
    im2 = ret1[i]
    im3 = ret2[i]
    images = [im1, im2, im3]
    
    total_width = 48*3
    max_height = 48

    new_im = Image.new('RGB', (total_width, max_height))

    x_offset = 0
    for im in images:
        new_im.paste(im, (x_offset,0))
        x_offset += im.size[0]

    new_im.save(Base_dir+'method2_img2emotion/comp/result_{}.jpg'.format(i))

basic_land shape:  (7, 144)
emotion landmarks shape:  (21, 144)
test landmarks shape:  (21, 144)
basic_land shape:  (7, 144)
emotion landmarks shape:  (21, 144)
test landmarks shape:  (21, 144)


In [0]:
for i in range(21):
    im1 = Image.open(Base_dir+'Testdata/{}.jpg'.format(str(i).zfill(2))).convert('RGB')
    im2 = Image.open(Base_dir+'emo_remix/result_{}.jpg'.format(i))
    im3 = Image.open(Base_dir+'response_basic/result_{}.jpg'.format(i))
    images = [im1, im2, im3]
    
    total_width = 48*3
    max_height = 48

    new_im = Image.new('RGB', (total_width, max_height))

    x_offset = 0
    for im in images:
        new_im.paste(im, (x_offset,0))
        x_offset += im.size[0]

    new_im.save(Base_dir+'final_result/result_{}.jpg'.format(i))


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!pip install face_recognition